In [97]:
import os
from langchain.chains import GraphCypherQAChain
from langchain_community.chat_models import ChatOllama
from langchain.prompts.prompt import PromptTemplate

In [80]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

In [81]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

movies_query = r"""
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [82]:
graph.query(movies_query)

[]

In [83]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {imdbRating: FLOAT, id: STRING, released: DATE, title: STRING}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [144]:
CYPHER_GENERATION_TEMPLATE = """
Task: Generate cypher statement to query a graph database.
Instructions:
Use only the provided relationships types and properties in the schema
Do not use any other relationship  types or properties that are not provided
Below are some examples for cypher queries.
Cypher examples:
# What is the genre of toy story?
Match (m:Movie)-[:IN_GENRE]->(g:Genre) where m.title = "Toy Story" return g;
# Give me 3 animated movies
Match (m:Movie) -[:IN_GENRE]->(g:Genre) where g.name = "Animation" return m limit 3;
# What is the release date of toy story?
Match (m:Movie) where m.title = "Toy Story" return m.released;
Schema:
{schema}
Note: Do not include any explanations or apologies in your response
Do not include any text except the generated Cypher statement
Do not respond to any questions that might ask you anything else other than to construct a Cypher query

The question is:
{question}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE)

In [145]:
model = ChatOllama(model="llama2")
chain = GraphCypherQAChain.from_llm(graph=graph, llm=model,cypher_prompt=CYPHER_GENERATION_PROMPT)

In [148]:
user_input = input("Input prompt")
try:
    if user_input:
        if (user_input.lower() in ("where", "what", "how", "why", "when")) and ("?" not in user_input.lower()):
            user_input = user_input + " " + "?" 
    
        # user_input = user_input.upper()
        response = chain.invoke({"query": user_input})
        
        if "don't know" in response['result']: 
            print("The movie might not be there in the database or give a better prompt")
        else:
            print(response['result'])
    else:
        print("Please provide a valid prompt")
except:
    print("Please provide a prompt relevant to movies")

Of course! Based on the information provided, here are 5 movies in the romance genre that are owned by CTL LLC and JANE STREET GROUP LLC:

1. Love Affair (released in 1994)
2. How to Make an American Quilt (released in 1995)
3. Frankie Starlight (released in 1995)
4. Goofy Movie, A (released in 1995)
5. Nina Takes a Lover (released in 1995)
